# Tutorial 11: Inflows 流入

This tutorial walks you through the process of introducing inflows of vehicles into a network. Inflows allow us to simulate open networks where vehicles may enter (and potentially exit) the network consanstly, such as a section of a highway or of an intersection.

本教程将带您了解将车辆引入网络的过程。流入使我们能够模拟车辆可能同时进入(和可能退出)网络的开放网络，例如高速公路的一段或十字路口。

The rest of this tutorial is organized as follows: 

- In [**section 1**](#1.-Creating-inflows-in-Flow), we introduce inflows and show how to create them into Flow. 我们介绍了流入，并展示了如何将它们创造为流。
- In [**section 2**](#2.-Running-simulations-with-inflows), we simulate the merge network in the presence of inflows.我们模拟了有流入时的合并网络。
- In [**section 3**](#3.-Customizing-inflows), we explain the different options you have to customize inflows.我们解释了不同的选择，你必须自定义流入。

## 1. Creating inflows in Flow 在flow中创建流入

For this tutorial, we will simulate inflows through a highway network with an entrance ramp (an on-merge). As we will see, the perturbations caused by the vehicles entering through the ramp leads to the formation of congested waves downstream in the main highway.
在本教程中，我们将模拟通过带有入口坡道(合并中的)的高速公路网的流入。正如我们将看到的，由车辆通过坡道进入引起的扰动导致在主要公路下游形成拥挤的波浪。

We begin by importing the merge network class provided by Flow.
我们首先导入Flow提供的合并网络类。

In [ ]:
from flow.networks import MergeNetwork

A schematic of the above network is displayed in the figure below. As we can see, the edges at the start of the main highway and of the on-merge are named `inflow_highway` and `inflow_merge` respectively. These names will be important when we begin creating our inflows, as we will need to specify by which edges the vehicles should enter the network.

上述网络的示意图如下图所示。正如我们所看到的，主高速公路的起点和合并中的边分别被命名为“inflow_highway”和“inflow_merge”。当我们开始创建流入时，这些名称将非常重要，因为我们需要指定车辆应该通过哪些边进入网络。

<img src="img/merge_scheme.png" width="750">

We also need to define the types of the vehicles that are placed in the network through our inflows. These types are string values that allow us to distinguish between vehicles. For instance, we could have two types of vehicles entering through the main highway, one for human-driven vehicles and one for RL-driven vehicles.

我们还需要定义通过流入放置在网络中的工具的类型。这些类型是允许我们区分车辆的字符串值。例如，我们可以有两种类型的车辆通过主要公路进入，一种是人驾驶的车辆，另一种是rl驾驶的车辆。

For this tutorial, we will only use one type of vehicles, with the vehicle identifier `human`:
在本教程中，我们将只使用一种类型的车辆，车辆标识符为“human”:

In [ ]:
from flow.core.params import VehicleParams
from flow.controllers import IDMController
from flow.core.params import SumoCarFollowingParams

# create an empty vehicles object
vehicles = VehicleParams()

# add some vehicles to this object of type "human"
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=20)

We have created a new type of vehicle, called `human`, and we directly inserted 20 vehicles of this type into the network. These vehicles will already be on the network when the simulation starts, contrary to the vehicles added by the inflow which will only start coming in the network after the simulation starts.

我们创造了一种新型的交通工具，叫做“人”，我们直接将20辆这种交通工具插入到网络中。当仿真开始时，这些车辆已经在网络上，与流入的车辆相反，流入的车辆只有在仿真开始后才开始进入网络。

Note that it is not necessary to add vehicles at the start. If you don't wish that to happen, you can set `num_vehicles=0`, which is the default value if you don't specify `num_vehicles` at all.

注意，没有必要在开始时添加车辆。如果您不希望这样，您可以设置' num_vehicles=0 '，这是默认值，如果您根本没有指定' num_vehicles '。

Next, we are ready to import and create an empty `InFlows` object.

In [ ]:
from flow.core.params import InFlows

inflow = InFlows()

The `InFlows` object is provided as an input during the network creation process via the `NetParams` parameter. Introducing these inflows into the network is handled by the backend network generation processes during instantiation of the network object.

“流入”对象是通过“NetParams”参数在网络创建过程中作为输入提供的。在网络对象实例化期间，后端网络生成过程处理这些流入网络的操作。


In order to add new inflows of vehicles of pre-defined types onto specific edges and lanes in the network, we use the `InFlows` object's `add` method. This function accepts the following parameters (more will be shown in section 3):

为了在网络的特定边缘和车道上添加预定义类型的车辆的新流入，我们使用“流入”对象的“添加”方法。该函数接受以下参数(更多参数将在第3节中显示):

* `veh_type`: the type of the vehicles the inflow will create (this must match one of the types set in the `VehicleParams` object),
流入将创建的车辆类型(这必须与‘VehicleParams’对象中设置的类型之一匹配)，
* `edge`: the name of the edge (in the network) where the inflow will insert vehicles,
流入将插入车辆的(网络中)边缘的名称，
* `vehs_per_hour`: the maximum number of vehicles entering from the edge per hour (this number may not be achievable due to congestion and safe driving behavior).
每小时从边缘进入的车辆的最大数量(由于交通堵塞和安全驾驶行为，这个数字可能无法达到)。

More options are shown in [**section 3**](#3.-Customizing-inflows).

We begin by creating an inflow of vehicles at a rate of 2000 vehicles per hour on the main highway:
我们首先在主要高速公路上以每小时2000辆的速度增加车辆流量:

In [ ]:
inflow.add(veh_type="human",
           edge="inflow_highway",
           vehs_per_hour=2000)

Next, we create a second inflow of vehicles on the inflow_merge lane at a lower rate of 100 vehicles per hour.
接下来，我们在inflow_merge车道上以每小时100辆的速度创建第二个车辆流入。

In [ ]:
inflow.add(veh_type="human",
           edge="inflow_merge",
           vehs_per_hour=100)

In the next section, we will add our inflows to our network and run a simulation to see them in action.

在下一节中，我们将把流入的数据添加到网络中，并运行一个模拟来观察它们的运行情况。

## 2. Running simulations with inflows 运行流入仿真

We are now ready to test our inflows in a simulation. Introducing these inflows into the network is handled by the backend network generation processes during the instantiation of the network object. To make this work, the `InFlows` object should be given as a parameter to the `NetParams` object, in addition to all other network-specific parameters.

我们现在准备在模拟中测试流入。在网络对象实例化期间，后端网络生成过程处理这些流入网络的操作。要实现这一功能，除了所有其他特定于网络的参数外，“流入”对象还应该作为“NetParams”对象的参数。

For the merge network, this is done as follows: 
对于合并网络，这是这样做的:

In [ ]:
from flow.networks.merge import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
# make the part of the highway after the merge longer
additional_net_params['post_merge_length'] = 350  
# make the number of lanes on the highway be just one
additional_net_params['highway_lanes'] = 1

net_params = NetParams(inflows=inflow,  # our inflows
                       additional_params=additional_net_params)

Finally, we create and start the simulation, following what is explained in tutorial 1.
最后，我们创建并启动模拟，遵循教程1中所解释的内容。

- _If the simulation in SUMO is going too fast, you can slow it down by sliding the "Delay" cursor from left to right._
如果相扑的模拟速度太快，你可以通过将“延时”光标从左向右滑动来降低速度
- _Don't worry about potential warnings that might come up in the log while runing the simulation._
不要担心在运行模拟时日志中可能出现的潜在警告

In [ ]:
from flow.core.params import SumoParams, EnvParams, InitialConfig
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

sim_params = SumoParams(render=True,
                         sim_step=0.2)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig()

flow_params = dict(
    exp_tag='merge-example',
    env_name=AccelEnv,
    network=MergeNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 10000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

<img src="img/merge_visual.png" width="100%">

Running this simulation, we can see that a large number of vehicles are entering from the main highway, while only a sparse number of vehicles are entering from the on-merge, as we specified in the inflows. Feel free to try different `vehs_per_hour` values so as to have different inflow rates.

运行这个模拟，我们可以看到大量的车辆从主高速公路进入，而只有稀疏的车辆从合并中进入，正如我们在流入中指定的。您可以随意尝试不同的' vehs_per_hour '值，以获得不同的流入率。

In the next section, we will see how to exploit the full capabilities of inflows.
在下一节中，我们将看到如何充分利用流入的能力。

## 3. Customizing inflows 定制流入

If you run the previous simulation carefully, you will see that the vehicles entering the network start with no speed. Besides, if you replace
如果你仔细运行前面的模拟，你会看到进入网络的车辆没有速度启动。此外，如果你替换
`additional_net_params['highway_lanes'] = 1` by `additional_net_params['highway_lanes'] = 2` in [**section 1**](#1.-Creating-inflows-in-Flow), thus making the highway two-lane-wide, you will see that vehicles only enter on the right lane. 
因此，使高速公路两车道宽，你会看到车辆只进入右车道。

In this section, we will see how to solve these issues, and how to customize inflows.

在本节中，我们将看到如何解决这些问题，以及如何定制流入。

We saw that you can create an inflow by doing the following:

我们看到，您可以通过以下操作来创建一个流入:

In [ ]:
inflow.add(veh_type="human",
           edge="inflow_highway",
           vehs_per_hour=2000)

However, this `add` method has a lot more parameters, which we will talk about now.
但是，这个“add”方法有更多的参数，我们现在将讨论这些参数。

Let's start with parameters that allow you to specify the inflow rate, i.e. how many vehicles the inflow will add into the network. 

让我们从允许您指定流入率的参数开始，即流入将向网络中添加多少车辆。

There are 3 parameters to do this: 

- `vehs_per_hour`: we have seen this one before, this is the number of vehicles that should enter the network, in vehicles per hour, equally spaced. For example, as there are $60 \times 60 = 3600$ seconds in one hour, setting this parameter to $\frac{3600}{5}=720$ will result in vehicles entering the network every $5$ seconds.

我们之前见过这个，这是进入网络的车辆数量，单位是每小时的车辆数，等间隔。例如，当1小时内有60×60=3600秒时，将该参数设置为36005=720，每5秒有车辆进入网络。

- `probability`: this is the probability (between 0 and 1) of a vehicle entering the network every second. For example, if we set this to $0.2$, then at each second of the simulation, a vehicle will enter the network with probability $\frac{1}{5}$.

概率:这是每秒钟车辆进入网络的概率(0到1之间)。例如，如果我们将这个设置为0.2，那么在每秒钟的模拟中，车辆进入网络的概率为15


- `period`: this is the time in seconds between two vehicles are inserted. For example, setting this to $5$ would result in vehicles entering the network every $5$ seconds (which is effectively the same as setting `vehs_per_hour` to $720$).

句点:这是插入两辆车之间的时间(以秒为单位)。例如，将这个设置为5

将导致车辆每5秒进入网络一次(这实际上等同于将vehs_per_hour设置为720)。

_Note that all these rates are **maximum** rates, meaning that if adding vehicles at the current rate would result in vehicles being too close to each other or colliding, then the rate will automatically be reduced._

请注意，所有这些速率都是最大速率，这意味着如果以当前速率添加车辆将导致车辆之间过于接近或碰撞，那么速率将自动降低。


Exactly **one** of these 3 parameters should be set, no more nor less. You can choose how you would rather have your vehicles enter the network. With `vehs_per_hour` and `period` (which are proportional to each other, use whichever is more convenient to define), vehicles will enter the network equally spaced, while the vehicles will be more randomly separated if you use `probability`.

这3个参数中的一个应该被设置，不能多也不能少。您可以选择您希望您的车辆如何进入网络。使用vehs_per_hour和period(它们是成比例的，使用更方便定义的方式)，车辆将以等间隔进入网络，而如果使用概率，则车辆将更随机地分隔开。

---

Now let's look into where and how fast vehicles enter the network.
现在让我们来看看车辆从哪里以及以多快的速度进入网络。

There are 2 parameters taking care of this:

- `depart_lane`: this parameter lets you specify in which lane vehicles are inserted when they enter the network on an edge consisting of several lanes. It should be a positive `int`, 0 being the rightmost lane. However most of the time, you don't want vehicles entering through only one lane (although you could create one inflow for each lane). That's why there are other options for this parameter, which are the following strings:
此参数允许您指定当车辆在由多条车道组成的边缘上进入网络时，将插入哪些车道。它应该是一个正的“int”，0是最右边的车道。然而，大多数情况下，您不希望车辆只通过一条车道进入(尽管您可以为每条车道创建一个流入)。这就是为什么这个参数有其他选项，这是以下字符串:

    - `"random"`: vehicles will enter on a random lane
    车辆将随机进入车道
    - `"free"`: vehicles will enter on the least occupied lane
    车辆将进入占用最少的车道
    - `"best"`: vehicles will enter on the `"free"` lane among those which allow the vehicle the longest ride without needing to change lane
    车辆将进入“自由”车道，这些车道允许车辆在不需要改变车道的情况下行驶最长的路程
    - `"first"`: vehicles will enter on the rightmost lane they can use
    车辆将进入他们可以使用的最右边的车道
    
  By default, `depart_lane` is set to `"free"`, which is why vehicles were only using the rightmost lane on the highway, if several lanes were available.
  默认情况下，' depart_lane '被设置为' free '，这就是为什么车辆只使用公路上最右边的车道，如果有多个车道可用的话。
  

- `depart_speed`: this parameter lets you specify the speed at which the vehicles will enter the network. It should be a positive `float`, in meters per second. If this speed is unsafe, the departure of the vehicles is delayed. Just like for `depart_lane`, there are other options for this parameter, which are the following strings:

该参数允许您指定车辆进入网络的速度。它应该是一个正的“浮点数”，单位是米/秒。如果这个速度是不安全的，车辆的出发就会延迟。和' depart_lane '一样，这个参数也有其他的选项，包括以下字符串:

    - `"random"`: vehicles enter the edge with a random speed between 0 and the speed limit on the edge. The entering speed may be adapted to ensure that a safe distance to the leading vehicle is kept
    车辆以0到边缘限速之间的随机速度进入边缘。可调整进入速度，以确保与前车保持安全距离
    - `"max"`: vehicle speeds at insertion will be adjusted to the maximum safe speed that allows insertion at the specified time to succeed
    车辆插入时的速度将调整到允许在指定时间插入成功的最大安全速度

   
By default, `depart_speed` is set to 0.默认为0

---

Finally, let's look at the rest of the parameters available:
最后，让我们看看其他可用的参数:

- `name` (`str`): a name for the inflow, which will also be used as a prefix for the ids of the vehicles created by it . This is set to `"flow"` by default.

流入的名称，它也将用作由它创建的车辆id的前缀。默认设置为“flow”。


- `begin` (`float`): the time of the simulation, in seconds, at which the inflow should start producing vehicles. This is set to 1 second by default, which is the minimum value (setting it to 0 could cause collisions with vehicles that are manually added into the network). 

模拟的时间，以秒为单位，开始产生车辆的时间。默认设置为1秒，这是最小值(设置为0可能会与手动添加到网络中的车辆发生冲突)。


- `end` (`float`): the time of the simulation, in seconds, at which the inflow should stop producing vehicles. This is set to 24 hours (86400 seconds) by default.

模拟的时间，以秒为单位，停止产生车辆的时间。默认设置为24小时(86400秒)。


- `number` (`int`): the number of vehicles that should be procuded by the inflow. This is set to `None` by default, which make the inflow keep producing vehicles indefinitely until `end` is reached. If this parameter is specified, the `end` parameter won't be used. Note that if this number is small, it might not be enforced accurately due to rounding up.

应由入境处提供的车辆数目。这是默认设置为“无”，这使得流入的车辆无限期地继续生产，直到“结束”。如果指定了此参数，则不会使用“end”参数。请注意，如果这个数字很小，则可能由于四舍五入而无法准确执行。


- `kwargs` (`dict`): you can specify additional parameters if you need to. These can include, for instance, a specific route for the vehicles to follow, an arrival speed, an arrival lane, or even a color for the vehicles, etc. For more information on all the available parameters, and more details on the existing parameters, see [here](https://sumo.dlr.de/wiki/Definition_of_Vehicles,_Vehicle_Types,_and_Routes#Vehicles_and_Routes).

如果需要，可以指定其他参数。这些可以包括，例如，车辆要遵循的特定路线，到达速度，到达车道，甚至车辆的颜色，等等。有关所有可用参数的更多信息，以及现有参数的更多细节，

---

Let us finish this section with a more complex example. This is what we want:

让我们用一个更复杂的例子来结束本节。这就是我们想要的:

- We will use the merge network, with no vehicles being manually pre-inserted into the network.

我们将使用合并网络，不需要手动预先将车辆插入网络。

- There will be 4 lanes on the main highway and 2 on the on-merge.

主要高速公路上将有4条车道，合并公路上将有2条。

- (1) Every hour, 10000 vehicles will enter the highway at maximum speed on a random lane, from the start of the simulation up until the end. These vehicles should be colored in white
每小时，10000辆汽车将以最高速度随机进入高速公路，从模拟的开始到结束。这些交通工具应该是白色的

- (2) Every two seconds, a vehicle will enter the on-merge with no speed, on the right lane, from the start of the simulation up until the end. These vehicles should be colored in green.
每隔两秒，就会有一辆车在没有速度的情况下，从模拟开始到结束，在右车道上进入“on-merge”。这些车辆应该涂成绿色。

- (3) Every second, a vehicle should enter with probability 0.1 on the left lane of the on-merge, with random speed. These vehicles should only start entering the network after the first minute of simulation time, and there should be at most 30 of them throughout the whole simulation. These vehicles should be colored in red.
每秒钟，车辆在合龙左侧车道以0.1概率进入，速度随机。这些车辆应该在模拟时间的第一分钟后才开始进入网络，在整个模拟过程中最多应该有30辆这样的车辆。这些车辆应该涂成红色。

_Note: for the colors, you will need to use the `kwargs` parameter._
注意:对于颜色，您需要使用“kwargs”参数
_Also, set_ `color_vehicles` _to_ `False` _in the simulation parameters so that the vehicles are not colored automatically according to their types._
另外，在仿真参数中设置_ ' color_vehicles ' _to_ ' False ' _以便车辆不会根据它们的类型自动着色。

The result should look something like this:

<img src="img/complex_merge_visual.png" width="100%"/>

You can try to do it yourself as an exercise if you want.

Here is a solution code:

In [ ]:
from flow.core.experiment import Experiment
from flow.core.params import NetParams, EnvParams, InitialConfig, InFlows, \
                             VehicleParams, SumoParams, SumoCarFollowingParams
from flow.controllers import IDMController
from flow.networks import MergeNetwork
from flow.networks.merge import ADDITIONAL_NET_PARAMS
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS



# create a vehicle type
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed"))


# create the inflows
inflows = InFlows()

# inflow for (1)
inflows.add(veh_type="human",
            edge="inflow_highway",
            vehs_per_hour=10000,
            depart_lane="random",
            depart_speed="random",
            color="white")

# inflow for (2)
inflows.add(veh_type="human",
            edge="inflow_merge",
            period=2,
            depart_lane=0,  # right lane
            depart_speed=0,
            color="green")

# inflow for (3)
inflows.add(veh_type="human",
           edge="inflow_merge",
           probability=0.1,
           depart_lane=1,  # left lane
           depart_speed="max",
           begin=60,  # 1 minute
           number=30,
           color="red")


# modify the network accordingly to instructions
# (the available parameters can be found in flow/networks/merge.py)
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
additional_net_params['post_merge_length'] = 350  # this is just for visuals
additional_net_params['highway_lanes'] = 4
additional_net_params['merge_lanes'] = 2


# setup and run the simulation
net_params = NetParams(inflows=inflows,
                       additional_params=additional_net_params)

sim_params = SumoParams(render=True,
                         sim_step=0.2)
#使车辆不会根据类型自动着色
sim_params.color_vehicles = False 

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig()

flow_params = dict(
    exp_tag='merge-example',
    env_name=AccelEnv,
    network=MergeNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 10000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)